### Get data

Using .pkl files because it hold the data the same way .mat does (the tactometer and vibration columns have very different amounts of data, and each file is its own day so I cant add it all to one big CSV)

In [ ]:
import os
import scipy.io
import pickle

input_folder = "WindTurbineData"
output_folder = "wind_turbine_pkl"
os.makedirs(output_folder, exist_ok=True)

for file in os.listdir(input_folder):
    if file.endswith(".mat"):
        path = os.path.join(input_folder, file)
        data = scipy.io.loadmat(path)

        vibration = data.get("vibration")
        tach = data.get("tach")

        # convert to flat list
        record = {
            "vibration": vibration.squeeze().tolist(),
            "tach": tach.squeeze().tolist()
        }

        # save
        base = os.path.splitext(file)[0]
        outpath = os.path.join(output_folder, base + ".pkl")
        with open(outpath, "wb") as f:
            pickle.dump(record, f)


the name of the mat files are the dates, I am renaming the files so its easier to use and adding the date as its own object like tact or vibration

In [ ]:
import os
import pandas as pd
from datetime import datetime

folder = "wind_turbine_pkl"  

# sort the files (will be sorted by date)
files = sorted([f for f in os.listdir(folder) if f.endswith(".pkl")])

# 
for i, filename in enumerate(files, start=1):
    old_path = os.path.join(folder, filename)

    # get timestamp from file name before its changed, and save as datetime 
    timestamp = filename.replace("data-", "").replace(".pkl", "")
    date_obj = datetime.strptime(timestamp, "%Y%m%dT%H%M%SZ")

    # load data and add the date
    with open(old_path, "rb") as f:
        data = pickle.load(f)
    data["date"] = date_obj

    new_name = f"wt_bearing{i}.pkl"
    new_path = os.path.join(folder, new_name)
    
    # save to new file
    with open(new_path, "wb") as f:
        pickle.dump(data, f)

    # remove original file
    os.remove(old_path)

In [ ]:
import pickle

with open("wind_turbine_pkl/wt_bearing1.pkl", "rb") as f:
    data = pickle.load(f)

for key, value in data.items():
    print(f"\nKey: {key}")
    
    if hasattr(value, "head"):
        print(value.head())  
    elif isinstance(value, (list, tuple)):
        print(value[:5])
    elif hasattr(value, "shape"):
        print(f"Shape: {value.shape}")
        print(value[:5])
    else:
        print(value)